# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [49]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [50]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils


In [51]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [52]:
x_train.shape
y_train.shape
x_test.shape
y_test.shape

(50000, 32, 32, 3)

(50000, 1)

(10000, 32, 32, 3)

(10000, 1)

In [53]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

In [54]:

n_examples = 50000

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])


In [55]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

In [56]:
X1_train.shape
X1_test.shape
X2_train.shape
X2_test.shape

(25000, 32, 32, 3)

(5000, 32, 32, 3)

(25000, 32, 32, 3)

(5000, 32, 32, 3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [57]:
Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes=10)


In [58]:
Y1_train.shape
Y1_test.shape
Y2_train.shape
Y2_test.shape

(25000, 5)

(5000, 5)

(25000, 10)

(5000, 10)

In [59]:
S1 = int(0.8 * X1_train.shape[0])
S2 = int(0.8 * X2_train.shape[0])

In [60]:

x1_valuation = X1_train[S1:]
x1_train = X1_train[:S1]
y1_valuation = Y1_train[S1:]
y1_train = Y1_train[:S1]

x2_valuation = X2_train[S2:]
x2_train = X2_train[:S2]
y2_valuation = Y2_train[S2:]
y2_train = Y2_train[:S2]

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [61]:
model = Sequential()

model.add(Convolution2D(32, 5, 5, input_shape=(32, 32, 3), activation='relu'))
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(8, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               102528    
__________

In [63]:
import time, datetime

start = datetime.datetime.now()
model1 = model.fit(x1_train, y1_train,
         nb_epoch=30,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_valuation, y1_valuation), verbose=2)

time_taken = datetime.datetime.now() - start
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)


Train on 20000 samples, validate on 5000 samples
Epoch 1/30
 - 61s - loss: 0.9395 - acc: 0.6157 - val_loss: 0.8847 - val_acc: 0.6446
Epoch 2/30
 - 64s - loss: 0.8904 - acc: 0.6387 - val_loss: 0.8542 - val_acc: 0.6640
Epoch 3/30
 - 63s - loss: 0.8575 - acc: 0.6538 - val_loss: 0.8723 - val_acc: 0.6620
Epoch 4/30
 - 63s - loss: 0.8316 - acc: 0.6652 - val_loss: 0.8143 - val_acc: 0.6750
Epoch 5/30
 - 63s - loss: 0.8014 - acc: 0.6813 - val_loss: 0.8442 - val_acc: 0.6724
Epoch 6/30
 - 62s - loss: 0.7812 - acc: 0.6907 - val_loss: 0.7848 - val_acc: 0.6972
Epoch 7/30
 - 62s - loss: 0.7594 - acc: 0.6986 - val_loss: 0.7578 - val_acc: 0.7062
Epoch 8/30
 - 61s - loss: 0.7348 - acc: 0.7105 - val_loss: 0.7596 - val_acc: 0.7104
Epoch 9/30
 - 61s - loss: 0.7101 - acc: 0.7241 - val_loss: 0.7326 - val_acc: 0.7168
Epoch 10/30
 - 61s - loss: 0.6776 - acc: 0.7376 - val_loss: 0.7290 - val_acc: 0.7214
Epoch 11/30
 - 62s - loss: 0.6670 - acc: 0.7401 - val_loss: 0.7534 - val_acc: 0.7088
Epoch 12/30
 - 61s - loss

## Achieved 80% accuracy at 22nd epoch. 
## Achieved a validation accuracy of 73.62% at 22 epochs. 
## The training time was 31 minutes and 13 seconds.

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [67]:
for l in model.layers[:6]:
    l.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [68]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               102528    
__________

In [69]:
start = datetime.datetime.now()
model2 = trans_model.fit(x2_train, y2_train, nb_epoch=25, shuffle=True, batch_size=100, validation_data=(x2_valuation, y2_valuation), verbose=2)
time_taken = datetime.datetime.now() - start
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
 - 13s - loss: 0.8948 - acc: 0.6612 - val_loss: 0.7392 - val_acc: 0.7234
Epoch 2/25
 - 12s - loss: 0.7498 - acc: 0.7192 - val_loss: 0.7006 - val_acc: 0.7400
Epoch 3/25
 - 13s - loss: 0.7116 - acc: 0.7374 - val_loss: 0.6690 - val_acc: 0.7516
Epoch 4/25
 - 13s - loss: 0.6766 - acc: 0.7441 - val_loss: 0.6580 - val_acc: 0.7588
Epoch 5/25
 - 12s - loss: 0.6490 - acc: 0.7561 - val_loss: 0.6451 - val_acc: 0.7594
Epoch 6/25
 - 12s - loss: 0.6257 - acc: 0.7642 - val_loss: 0.6220 - val_acc: 0.7726
Epoch 7/25
 - 13s - loss: 0.5970 - acc: 0.7752 - val_loss: 0.6226 - val_acc: 0.7750
Epoch 8/25
 - 13s - loss: 0.5812 - acc: 0.7811 - val_loss: 0.6072 - val_acc: 0.7790
Epoch 9/25
 - 13s - loss: 0.5580 - acc: 0.7913 - val_loss: 0.5957 - val_acc: 0.7856
Epoch 10/25
 - 13s - loss: 0.5406 - acc: 0.7996 - val_loss: 0.5994 - val_acc: 0.7800
Epoch 11/25
 - 13s - loss: 0.5268 - acc: 0.8030 - val_loss: 0.5945 - val_acc: 0.7858
Epoch 12/25
 - 13s - loss

## Used transfer learning to expedite the training process.
## 86% accuracy is acheived at 25th Epoch. 
## validadation accuracy of 78.82% is acheived at 25 epochs. 
## The training time has been reduced to 5 minnutes and 7 seconds.

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [70]:
from sklearn.datasets import fetch_20newsgroups

In [71]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [72]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [73]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [74]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [75]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [76]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.95)
train_data = vectorizer.fit_transform(twenty_train.data)
test_data = vectorizer.transform(twenty_test.data)
train_data = train_data.todense()
test_data = test_data.todense()
train_labels = twenty_train.target
test_labels = twenty_test.target

print(train_data.shape)
print(test_data.shape)

print(train_data, train_labels, test_data, test_labels)

(2257, 2384)
(1502, 2384)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.01186874 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] [1 1 3 ... 2 2 2] [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.06224145 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] [2 2 2 ... 2 2 1]


In [80]:
cvec = CountVectorizer()
train_data_features=cvec.fit(twenty_train.data)
cvecdata= cvec.transform(twenty_train.data)
df=pd.DataFrame(cvecdata.todense(),columns=cvec.get_feature_names())

print("Features : ", df)

Features :        00  000  0000  0000001200  000005102000  0001  000100255pixel  00014  \
0      0    0     0           0             0     0               0      0   
1      0    0     0           0             0     0               0      0   
2      0    0     0           0             0     0               0      0   
3      0    0     0           0             0     0               0      0   
4      1    0     0           0             0     0               0      0   
5      1    0     0           0             0     0               0      0   
6      0    0     0           0             0     0               0      0   
7      0    0     0           0             0     0               0      0   
8      0    0     0           0             0     0               0      0   
9      0    0     0           0             0     0               0      0   
10     0    0     0           0             0     0               0      0   
11     0    0     0           0             0     0 

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [81]:
from sklearn.linear_model import LogisticRegression
y=twenty_train.target
model=LogisticRegression()
log_model=model.fit(df,y)
log_model.score(df,y)

1.0

In [82]:
X_test=cvec.transform(twenty_test.data)
y_test=twenty_test.target
log_model.score(X_test,y_test)

0.9007989347536618

In [83]:
test_pred = model.predict(X_test)
from sklearn import metrics
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.93      0.81      0.86       319
           1       0.88      0.96      0.92       389
           2       0.93      0.84      0.88       396
           3       0.88      0.98      0.93       398

   micro avg       0.90      0.90      0.90      1502
   macro avg       0.90      0.90      0.90      1502
weighted avg       0.90      0.90      0.90      1502

[[258   8  13  40]
 [  5 373   9   2]
 [ 12  40 333  11]
 [  3   3   3 389]]


### Train accuracy is 100%
### Test accuracy is 90%